In [34]:
# mac m1 : brew install cmake libomp
# !uv add xgboost

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error  # 루트 제곱 평균 오차
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.preprocessing import StandardScaler  # 평균 0, 표준편차 1
from sklearn.preprocessing import MinMaxScaler  # 백분위
from sklearn.preprocessing import RobustScaler

In [2]:
file_url = "https://media.githubusercontent.com/media/musthave-ML10/data_source/main/dating.csv"
data = pd.read_csv(file_url)
data.shape

(8378, 39)

In [3]:
pd.options.display.max_columns = 40

In [4]:
data.head()

,has_null,gender,age,age_o,race,race_o,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,attractive_o,sincere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,like,guess_prob_liked,met,match
0,0,female,21.0,27.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,8.0,8.0,8.0,8.0,6.0,15.0,20.0,20.0,15.0,15.0,15.0,6.0,9.0,7.0,7.0,6.0,5.0,0.14,3.0,2.0,7.0,6.0,0.0,0
1,0,female,21.0,22.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,8.0,10.0,7.0,7.0,5.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,8.0,7.0,8.0,5.0,6.0,0.54,3.0,2.0,7.0,5.0,1.0,0
2,1,female,21.0,22.0,Asian/PacificIslander/Asian-American,Asian/PacificIslander/Asian-American,2.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,10.0,10.0,10.0,10.0,10.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,8.0,9.0,8.0,5.0,7.0,0.16,3.0,2.0,7.0,NaN,1.0,1
3,0,female,21.0,23.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,8.0,9.0,8.0,9.0,8.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,6.0,8.0,7.0,6.0,8.0,0.61,3.0,2.0,7.0,6.0,0.0,1
4,0,female,21.0,24.0,Asian/PacificIslander/Asian-American,Latino/HispanicAmerican,2.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,7.0,9.0,6.0,9.0,7.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,6.0,7.0,7.0,6.0,6.0,0.21,3.0,2.0,6.0,6.0,0.0,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 39 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   has_null                       8378 non-null   int64  
 1   gender                         8378 non-null   object 
 2   age                            8283 non-null   float64
 3   age_o                          8274 non-null   float64
 4   race                           8315 non-null   object 
 5   race_o                         8305 non-null   object 
 6   importance_same_race           8299 non-null   float64
 7   importance_same_religion       8299 non-null   float64
 8   pref_o_attractive              8289 non-null   float64
 9   pref_o_sincere                 8289 non-null   float64
 10  pref_o_intelligence            8289 non-null   float64
 11  pref_o_funny                   8280 non-null   float64
 12  pref_o_ambitious               8271 non-null   f

In [6]:
round(data.describe(), 2)

,has_null,age,age_o,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,attractive_o,sincere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,like,guess_prob_liked,met,match
count,8378.00,8283.00,8274.00,8299.00,8299.00,8289.00,8289.00,8289.00,8280.00,8271.00,8249.00,8166.00,8091.00,8072.00,8018.00,7656.00,7302.00,8299.00,8299.00,8299.00,8289.00,8279.00,8257.00,8176.00,8101.00,8082.00,8028.00,7666.00,7311.00,8220.00,8277.00,1800.00,8138.00,8069.00,8003.00,8378.00
mean,0.87,26.36,26.36,3.78,3.65,22.50,17.40,20.27,17.46,10.69,11.85,6.19,7.18,7.37,6.40,6.78,5.47,22.51,17.40,20.27,17.46,10.68,11.85,6.19,7.18,7.37,6.40,6.78,5.47,0.20,5.53,5.57,6.13,5.21,0.05,0.16
std,0.33,3.57,3.56,2.85,2.81,12.57,7.04,6.78,6.09,6.13,6.36,1.95,1.74,1.55,1.95,1.79,2.16,12.59,7.05,6.78,6.09,6.12,6.36,1.95,1.74,1.55,1.95,1.79,2.16,0.30,1.73,4.76,1.84,2.13,0.28,0.37
min,0.00,18.00,18.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.83,1.00,0.00,0.00,0.00,0.00,0.00
25%,1.00,24.00,24.00,1.00,1.00,15.00,15.00,17.39,15.00,5.00,9.52,5.00,6.00,6.00,5.00,6.00,4.00,15.00,15.00,17.39,15.00,5.00,9.52,5.00,6.00,6.00,5.00,6.00,4.00,-0.02,5.00,2.00,5.00,4.00,0.00,0.00
50%,1.00,26.00,26.00,3.00,3.00,20.00,18.37,20.00,18.00,10.00,10.64,6.00,7.00,7.00,7.00,7.00,6.00,20.00,18.18,20.00,18.00,10.00,10.64,6.00,7.00,7.00,7.00,7.00,6.00,0.21,6.00,4.00,6.00,5.00,0.00,0.00
75%,1.00,28.00,28.00,6.00,6.00,25.00,20.00,23.81,20.00,15.00,16.00,8.00,8.00,8.00,8.00,8.00,7.00,25.00,20.00,23.81,20.00,15.00,16.00,8.00,8.00,8.00,8.00,8.00,7.00,0.43,7.00,8.00,7.00,7.00,0.00,0.00
max,1.00,55.00,55.00,10.00,10.00,100.00,60.00,50.00,50.00,53.00,30.00,10.50,10.00,10.00,11.00,10.00,10.00,100.00,60.00,50.00,50.00,53.00,30.00,10.00,10.00,10.00,10.00,10.00,10.00,0.91,10.00,20.00,10.00,10.00,8.00,1.00


# 전처리 : 결측치

In [41]:
data.isna().mean() * 100

has_null                          0.000000
gender                            0.000000
age                               1.133922
age_o                             1.241346
race                              0.751969
race_o                            0.871330
importance_same_race              0.942946
importance_same_religion          0.942946
pref_o_attractive                 1.062306
pref_o_sincere                    1.062306
pref_o_intelligence               1.062306
pref_o_funny                      1.169730
pref_o_ambitious                  1.277154
pref_o_shared_interests           1.539747
attractive_o                      2.530437
sincere_o                         3.425639
intelligence_o                    3.652423
funny_o                           4.296968
ambitous_o                        8.617809
shared_interests_o               12.843161
attractive_important              0.942946
sincere_important                 0.942946
intellicence_important            0.942946
funny_impor

In [42]:
data.columns

Index(['has_null', 'gender', 'age', 'age_o', 'race', 'race_o',
       'importance_same_race', 'importance_same_religion', 'pref_o_attractive',
       'pref_o_sincere', 'pref_o_intelligence', 'pref_o_funny',
       'pref_o_ambitious', 'pref_o_shared_interests', 'attractive_o',
       'sincere_o', 'intelligence_o', 'funny_o', 'ambitous_o',
       'shared_interests_o', 'attractive_important', 'sincere_important',
       'intellicence_important', 'funny_important', 'ambtition_important',
       'shared_interests_important', 'attractive_partner', 'sincere_partner',
       'intelligence_partner', 'funny_partner', 'ambition_partner',
       'shared_interests_partner', 'interests_correlate',
       'expected_happy_with_sd_people', 'expected_num_interested_in_me',
       'like', 'guess_prob_liked', 'met', 'match'],
      dtype='object')

In [7]:
data = data.dropna(
    subset=[
        "pref_o_attractive",
        "pref_o_sincere",
        "pref_o_intelligence",
        "pref_o_funny",
        "pref_o_ambitious",
        "pref_o_shared_interests",
        "attractive_o",
        "sincere_o",
        "intelligence_o",
        "funny_o",
        "ambitous_o",
        "shared_interests_o",
        "attractive_important",
        "sincere_important",
        "intellicence_important",
        "funny_important",
        "ambtition_important",
        "shared_interests_important",
    ]
)

In [8]:
data = data.fillna(-99)

# 전처리 : 피처 엔지니어링

In [9]:
def age_gap(x):
  if x["age"] == -99:
    return -99
  elif x["age_o"] == -99:
    return -99
  elif x["gender"] == "female":
    return x["age_o"] - x["age"]
  else:
    return x["age"] - x["age_o"]

In [10]:
print(data.loc[9, ["gender", "age", "age_o"]])
print(age_gap(data.loc[9]))

gender    female
age         21.0
age_o       24.0
Name: 9, dtype: object
3.0


In [11]:
data["age_gap"] = data.apply(age_gap, axis=1)

In [48]:
data["age_gap"].sort_values(ascending=False)

3337    20.0
2804    20.0
3336    20.0
2784    20.0
3332    19.0
        ... 
7476   -99.0
8205   -99.0
7486   -99.0
7490   -99.0
8139   -99.0
Name: age_gap, Length: 6828, dtype: float64

In [12]:
data["age_gap_abs"] = abs(data["age_gap"])

In [50]:
data.head(2)

,has_null,gender,age,age_o,race,race_o,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,attractive_o,sincere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,...,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,like,guess_prob_liked,met,match,age_gap,age_gap_abs
0,0,female,21.0,27.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,8.0,8.0,8.0,8.0,6.0,...,20.0,20.0,15.0,15.0,15.0,6.0,9.0,7.0,7.0,6.0,5.0,0.14,3.0,2.0,7.0,6.0,0.0,0,6.0,6.0
1,0,female,21.0,22.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,8.0,10.0,7.0,7.0,5.0,...,20.0,20.0,15.0,15.0,15.0,7.0,8.0,7.0,8.0,5.0,6.0,0.54,3.0,2.0,7.0,5.0,1.0,0,1.0,1.0


In [13]:
def same_race(x):
    if x["race"] == -99:
        return -99
    elif x["race_o"] == -99:
        return -99
    elif x["race"] == x["race_o"]:
        return 1
    else:
        return 0

In [14]:
data["same_race"] = data.apply(same_race, axis=1)

In [15]:
def same_race_point(x):
    if x["same_race"] == -99:
        return -99
    else:
        return x["same_race"] * x["importance_same_race"]

In [16]:
data["same_race_point"] = data.apply(same_race_point, axis=1)

In [17]:
def rating(data, importance, score):
  if data[importance] == -99:
    return -99
  elif data[score] == -99:
    return -99
  else:
    return data[importance] * data[score]

In [19]:
data.columns[26:32]

Index(['attractive_partner', 'sincere_partner', 'intelligence_partner',
       'funny_partner', 'ambition_partner', 'shared_interests_partner'],
      dtype='object')

In [18]:
# 상대방의 중요도
partner_imp = data.columns[8:14]
# 본인에 대한 상대방의 평가
partner_rate_me = data.columns[14:20]
# 본인의 중요도
my_imp = data.columns[20:26]
# 상대방에 대한 본인의 평가
my_rate_partner = data.columns[26:32]

In [20]:
new_label_partner = ['attractive_p', 'sincere_p', 'intelligence_p',
    'funny_p', 'ambition_p', 'shared_interests_p']
new_label_me = ['attractive_m', 'sincere_m', 'intelligence_m',
    'funny_m', 'ambition_m', 'shared_interests_m']

In [21]:
for idx, val1, val2 in zip(new_label_partner, partner_imp, partner_rate_me):
  print(idx, " & ", val1, " & ", val2)

attractive_p  &  pref_o_attractive  &  attractive_o
sincere_p  &  pref_o_sincere  &  sincere_o
intelligence_p  &  pref_o_intelligence  &  intelligence_o
funny_p  &  pref_o_funny  &  funny_o
ambition_p  &  pref_o_ambitious  &  ambitous_o
shared_interests_p  &  pref_o_shared_interests  &  shared_interests_o


In [22]:
for idx, val1, val2 in zip(new_label_me, my_imp, my_rate_partner):
    print(idx, " & ", val1, " & ", val2)

attractive_m  &  attractive_important  &  attractive_partner
sincere_m  &  sincere_important  &  sincere_partner
intelligence_m  &  intellicence_important  &  intelligence_partner
funny_m  &  funny_important  &  funny_partner
ambition_m  &  ambtition_important  &  ambition_partner
shared_interests_m  &  shared_interests_important  &  shared_interests_partner


In [23]:
for idx, val1, val2 in zip(new_label_partner, partner_imp, partner_rate_me):
    data[idx] = data.apply(lambda x: rating(x, val1, val2), axis=1)

In [24]:
for idx, val1, val2 in zip(new_label_me, my_imp, my_rate_partner):
    data[idx] = data.apply(lambda x: rating(x, val1, val2), axis=1)

In [25]:
data.head(2)

,has_null,gender,age,age_o,race,race_o,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,attractive_o,sincere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,...,like,guess_prob_liked,met,match,age_gap,age_gap_abs,same_race,same_race_point,attractive_p,sincere_p,intelligence_p,funny_p,ambition_p,shared_interests_p,attractive_m,sincere_m,intelligence_m,funny_m,ambition_m,shared_interests_m
0,0,female,21.0,27.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,8.0,8.0,8.0,8.0,6.0,...,7.0,6.0,0.0,0,6.0,6.0,0,0.0,210.0,160.0,160.0,160.0,0.0,30.0,90.0,180.0,140.0,105.0,90.0,75.0
1,0,female,21.0,22.0,Asian/PacificIslander/Asian-American,European/Caucasian-American,2.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,8.0,10.0,7.0,7.0,5.0,...,7.0,5.0,1.0,0,1.0,1.0,0,0.0,420.0,0.0,0.0,280.0,0.0,0.0,105.0,160.0,140.0,120.0,75.0,90.0


In [26]:
data = pd.get_dummies(data, columns=["gender", "race", "race_o"], drop_first=True)

# 모델 학습 및 평가

In [27]:
X_train, X_test, y_train, y_test = train_test_split(data.drop("match", axis=1), 
                                                    data["match"], test_size=0.3, random_state=20)

In [29]:
help(xgb.XGBClassifier)

Help on class XGBClassifier in module xgboost.sklearn:

class XGBClassifier(sklearn.base.ClassifierMixin, XGBModel)
 |  XGBClassifier(*, objective: Union[str, xgboost.sklearn._SklObjWProto, Callable[[Any, Any], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'binary:logistic', **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  See :doc:`/python/sklearn_estimator` for more information.
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : Optional[int]
 |          Number of boosting rounds.
 |  
 |      max_depth :  typing.Optional[int]
 |  
 |          Maximum tree depth for base learners.
 |  
 |      max_leaves : typing.Optional[int]
 |  
 |          Maximum number of leaves; 0 indicates no limit.
 |  
 |      max_bin : typing.Optional[int]
 |  
 |          If using histogram-based algorithm, maximum number of bins per feature
 |  
 |      grow_policy : typing.Optional[str]
 |  
 |          Tree growing policy.
 |  
 |   

In [30]:
model = xgb.XGBClassifier(n_estimators=500, max_depth=5, random_state=20)
model.fit(X_train, y_train)
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [31]:
print("train score: ", accuracy_score(y_train, train_pred), " test score: ", accuracy_score(y_test, test_pred))

train score:  1.0  test score:  0.8521229868228404


In [32]:
confusion_matrix(y_test, test_pred)

array([[1593,   93],
       [ 210,  153]])

In [34]:
from sklearn.metrics import auc, roc_auc_score, f1_score, classification_report
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91      1686
           1       0.62      0.42      0.50       363

    accuracy                           0.85      2049
   macro avg       0.75      0.68      0.71      2049
weighted avg       0.84      0.85      0.84      2049



In [ ]:
# help(roc_auc_score)
roc_auc_score(y_test, test_pred)

np.float64(0.6831637304785153)

In [35]:
data.match.value_counts()

match
0    5650
1    1178
Name: count, dtype: int64

In [41]:
data.columns

Index(['has_null', 'age', 'age_o', 'importance_same_race',
       'importance_same_religion', 'pref_o_attractive', 'pref_o_sincere',
       'pref_o_intelligence', 'pref_o_funny', 'pref_o_ambitious',
       'pref_o_shared_interests', 'attractive_o', 'sincere_o',
       'intelligence_o', 'funny_o', 'ambitous_o', 'shared_interests_o',
       'attractive_important', 'sincere_important', 'intellicence_important',
       'funny_important', 'ambtition_important', 'shared_interests_important',
       'attractive_partner', 'sincere_partner', 'intelligence_partner',
       'funny_partner', 'ambition_partner', 'shared_interests_partner',
       'interests_correlate', 'expected_happy_with_sd_people',
       'expected_num_interested_in_me', 'like', 'guess_prob_liked', 'met',
       'match', 'age_gap', 'age_gap_abs', 'same_race', 'same_race_point',
       'attractive_p', 'sincere_p', 'intelligence_p', 'funny_p', 'ambition_p',
       'shared_interests_p', 'attractive_m', 'sincere_m', 'intelligence_m'

In [43]:
data_subset = data.loc[:,[
    "interests_correlate",
    "expected_happy_with_sd_people",
    "expected_num_interested_in_me",
    "like",
    "guess_prob_liked",
    "met",
    "match",
    "age_gap_abs",
    "same_race",
    "same_race_point",
    "attractive_p",
    "sincere_p",
    "intelligence_p",
    "funny_p",
    "ambition_p",
    "shared_interests_p",
    "attractive_m",
    "sincere_m",
    "intelligence_m",
    "funny_m",
    "ambition_m",
    "shared_interests_m",
    "gender_male"]
]

In [46]:
print(data_subset.shape, data.shape)
data_subset.head()

(6828, 23) (6828, 61)


,interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,like,guess_prob_liked,met,match,age_gap_abs,same_race,same_race_point,attractive_p,sincere_p,intelligence_p,funny_p,ambition_p,shared_interests_p,attractive_m,sincere_m,intelligence_m,funny_m,ambition_m,shared_interests_m,gender_male
0,0.14,3.0,2.0,7.0,6.0,0.0,0,6.0,0,0.0,210.0,160.0,160.0,160.0,0.0,30.0,90.0,180.0,140.0,105.0,90.0,75.0,False
1,0.54,3.0,2.0,7.0,5.0,1.0,0,1.0,0,0.0,420.0,0.0,0.0,280.0,0.0,0.0,105.0,160.0,140.0,120.0,75.0,90.0,False
2,0.16,3.0,2.0,7.0,-99.0,1.0,1,1.0,1,2.0,190.0,180.0,190.0,180.0,140.0,120.0,75.0,160.0,180.0,120.0,75.0,105.0,False
3,0.61,3.0,2.0,7.0,6.0,0.0,1,2.0,0,0.0,210.0,40.0,135.0,320.0,45.0,40.0,105.0,120.0,160.0,105.0,90.0,120.0,False
4,0.21,3.0,2.0,6.0,6.0,0.0,1,3.0,0,0.0,240.0,70.0,180.0,60.0,90.0,140.0,75.0,120.0,140.0,105.0,90.0,90.0,False


In [47]:
data_subset.describe()

,interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,like,guess_prob_liked,met,match,age_gap_abs,same_race,same_race_point,attractive_p,sincere_p,intelligence_p,funny_p,ambition_p,shared_interests_p,attractive_m,sincere_m,intelligence_m,funny_m,ambition_m,shared_interests_m
count,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000,6828.000000
mean,0.197897,5.277241,-76.557118,4.809710,3.078500,-2.760398,0.172525,4.084359,0.400557,1.649971,139.214811,124.884739,149.351340,111.414528,74.515018,64.558802,136.858281,121.827668,145.449943,106.644815,62.795075,46.923802
std,0.303646,5.343348,43.020706,11.998781,14.971167,16.461344,0.377864,7.355954,0.490047,2.752296,95.452195,59.020978,57.690364,52.384330,48.661738,44.096458,95.538338,65.006443,67.507786,62.310593,64.531511,66.847174
min,-0.830000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000
25%,-0.020000,5.000000,-99.000000,5.000000,4.000000,0.000000,0.000000,1.000000,0.000000,0.000000,80.000000,90.000000,120.000000,75.000000,40.000000,30.000000,80.000000,84.975000,120.000000,72.000000,30.000000,20.000000
50%,0.220000,6.000000,-99.000000,6.000000,5.000000,0.000000,0.000000,3.000000,0.000000,0.000000,120.000000,121.730000,148.960000,106.680000,70.000000,60.000000,120.000000,122.500000,147.900000,105.000000,70.000000,50.000000
75%,0.430000,7.000000,-99.000000,7.000000,7.000000,0.000000,0.000000,5.000000,1.000000,2.000000,168.000000,160.000000,180.000000,140.000000,105.000000,92.357500,167.580000,160.000000,180.000000,140.000000,105.000000,90.000000
max,0.910000,10.000000,20.000000,10.000000,10.000000,8.000000,1.000000,99.000000,1.000000,10.000000,950.000000,423.000000,500.000000,500.000000,477.000000,270.000000,950.000000,423.000000,500.000000,500.000000,477.000000,270.000000


In [55]:
data.corr()

,has_null,age,age_o,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,attractive_o,sincere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,attractive_important,sincere_important,intellicence_important,...,sincere_p,intelligence_p,funny_p,ambition_p,shared_interests_p,attractive_m,sincere_m,intelligence_m,funny_m,ambition_m,shared_interests_m,gender_male,race_Black/AfricanAmerican,race_European/Caucasian-American,race_Latino/HispanicAmerican,race_Other,race_o_Black/AfricanAmerican,race_o_European/Caucasian-American,race_o_Latino/HispanicAmerican,race_o_Other
has_null,1.000000,0.062016,0.064395,-0.035420,-0.073773,-0.049984,0.027393,-0.122599,0.006644,0.074307,0.099513,-0.001666,0.001127,-0.003635,0.024536,-0.006080,0.009989,-0.055170,0.007146,-0.092156,...,0.025671,-0.101946,0.017791,0.065648,0.080643,-0.048446,-0.020297,-0.097017,0.001073,-0.041995,-0.081628,-0.001795,0.028829,-0.038054,-0.002149,-0.025006,-0.022706,-0.040659,0.001975,-0.001273
age,0.062016,1.000000,0.033254,-0.037270,0.013091,-0.041563,0.009659,-0.004493,-0.023868,0.068613,0.028114,-0.061686,0.008046,0.045437,-0.021383,0.013506,0.022956,0.021188,0.001626,0.028108,...,0.009530,0.022402,-0.033414,0.062219,0.026104,0.032645,-0.015241,0.008128,-0.040345,-0.034187,-0.041814,0.079275,0.029014,-0.028214,0.033554,0.006580,0.003874,-0.001665,0.016922,0.003027
age_o,0.064395,0.033254,1.000000,-0.005882,0.027995,0.038344,-0.004369,0.030696,-0.064932,-0.012869,-0.027712,0.016744,-0.021520,-0.038998,0.017607,-0.050788,0.023683,-0.050528,0.011238,-0.004831,...,-0.019826,0.007800,-0.037810,-0.019439,-0.001289,-0.063172,0.014622,0.013169,-0.028893,0.039454,0.010687,-0.093682,-0.000093,-0.001257,0.016559,0.002928,0.022448,-0.029771,0.045828,0.007385
importance_same_race,-0.035420,-0.037270,-0.005882,1.000000,0.432889,0.005982,-0.000921,-0.014226,-0.000760,-0.008036,0.018257,-0.009607,-0.014083,-0.030556,-0.008258,-0.003341,0.000531,-0.008584,-0.082632,0.002967,...,-0.006805,-0.032269,-0.002402,-0.009203,0.011316,-0.040594,-0.045834,0.009598,-0.036684,0.094588,0.013351,-0.118210,-0.037528,0.172871,-0.118238,-0.072351,-0.007037,0.015239,-0.001346,0.006652
importance_same_religion,-0.073773,0.013091,0.027995,0.432889,1.000000,0.059273,-0.003064,-0.003597,-0.016955,-0.069661,-0.018043,-0.027066,-0.023074,-0.033821,-0.029764,-0.038813,-0.014628,-0.145842,-0.001526,0.085156,...,-0.012819,-0.021038,-0.031132,-0.075516,-0.019128,-0.125686,0.020624,0.073617,-0.065541,0.084748,-0.017591,-0.199090,-0.073378,0.144369,-0.033522,-0.013786,-0.003483,0.024213,-0.004601,-0.008919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
race_Other,-0.025006,0.006580,0.002928,-0.072351,-0.013786,0.006192,0.005570,-0.021538,0.009406,-0.008837,0.006022,-0.025136,0.018683,0.008199,0.018851,0.005610,0.007840,0.005025,0.043599,0.013808,...,0.015248,-0.015100,0.019280,-0.001272,0.006596,0.018417,0.044450,0.030538,0.045419,-0.011446,0.012597,0.012940,-0.062241,-0.300492,-0.077499,1.000000,-0.011390,-0.018852,0.005710,0.017644
race_o_Black/AfricanAmerican,-0.022706,0.003874,0.022448,-0.007037,-0.003483,0.002154,-0.011561,-0.031123,-0.012891,0.062548,-0.025696,0.036652,0.095376,0.106844,0.078506,0.103421,0.049970,-0.013420,0.005961,0.011540,...,0.039893,0.027931,0.039419,0.102160,0.014280,-0.024517,0.015838,0.004573,0.027469,0.009697,0.012993,0.043178,-0.005072,-0.000187,0.003428,-0.011390,1.000000,-0.259712,-0.064838,-0.060690
race_o_European/Caucasian-American,-0.040659,-0.001665,-0.029771,0.015239,0.024213,0.075384,-0.176393,0.117479,0.106545,-0.058822,-0.111047,0.023602,0.035286,0.037708,-0.001545,0.030210,-0.023428,-0.021631,0.004365,0.016899,...,-0.119714,0.121411,0.078067,-0.030691,-0.110739,0.033454,-0.013546,0.004287,0.011882,-0.002978,-0.005673,-0.076004,0.007360,0.027714,-0.0

In [51]:
rs_scaler = RobustScaler()
data_subset_scaled = rs_scaler.fit_transform(data_subset)
data_subset_scaled = pd.DataFrame(data_subset_scaled, columns=data_subset.columns)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(
    data_subset_scaled.drop("match", axis=1), data_subset["match"], test_size=0.3, random_state=20
)
model = xgb.XGBClassifier(n_estimators=1000, max_depth=5, random_state=20)
model.fit(X_train, y_train)
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [63]:
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91      1686
           1       0.58      0.39      0.47       363

    accuracy                           0.84      2049
   macro avg       0.73      0.67      0.69      2049
weighted avg       0.83      0.84      0.83      2049



# 하이퍼파라미터 최적화 : 그리드 서치

In [64]:
help(xgb.XGBClassifier)

Help on class XGBClassifier in module xgboost.sklearn:

class XGBClassifier(sklearn.base.ClassifierMixin, XGBModel)
 |  XGBClassifier(*, objective: Union[str, xgboost.sklearn._SklObjWProto, Callable[[Any, Any], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'binary:logistic', **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  See :doc:`/python/sklearn_estimator` for more information.
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : Optional[int]
 |          Number of boosting rounds.
 |  
 |      max_depth :  typing.Optional[int]
 |  
 |          Maximum tree depth for base learners.
 |  
 |      max_leaves : typing.Optional[int]
 |  
 |          Maximum number of leaves; 0 indicates no limit.
 |  
 |      max_bin : typing.Optional[int]
 |  
 |          If using histogram-based algorithm, maximum number of bins per feature
 |  
 |      grow_policy : typing.Optional[str]
 |  
 |          Tree growing policy.
 |  
 |   

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop("match", axis=1),
    data["match"],
    test_size=0.3,
    random_state=20,
)
parameters = {
  "learning_rate" : [0.01, 0.1, 0.3, 0.5],
  "max_depth" : [5, 7, 9, 10],
  "subsample" : [0.3, 0.5, 0.7, 1],
  "n_estimators" : [300, 500, 700, 1000]
}
model = xgb.XGBClassifier()

In [74]:
from sklearn.model_selection import GridSearchCV
gs_model = GridSearchCV(model, parameters, n_jobs=-1, scoring="f1", cv=5)

In [75]:
gs_model.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, feature_weights=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=Non...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.3, 0.5],
                         'max_depth': [5, 7, 9, 10],
                         'n_estimators': [300, 500, 700, 1000],
                         'subsample': [0.3, 0.5, 0.7, 1]},
             scoring='f1')

In [76]:
gs_model.best_params_

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 1000, 'subsample': 1}

In [77]:
test_pred = gs_model.predict(X_test)

In [78]:
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92      1686
           1       0.64      0.44      0.52       363

    accuracy                           0.86      2049
   macro avg       0.76      0.69      0.72      2049
weighted avg       0.84      0.86      0.85      2049



In [80]:
gs_model.predict_proba(X_test)

array([[9.9997193e-01, 2.8066455e-05],
       [9.9580717e-01, 4.1928301e-03],
       [1.8838090e-01, 8.1161910e-01],
       ...,
       [9.9961281e-01, 3.8718976e-04],
       [9.9744582e-01, 2.5541929e-03],
       [9.9455273e-01, 5.4472983e-03]], shape=(2049, 2), dtype=float32)

In [81]:
!uv add lightgbm

Resolved 121 packages in 6ms
Audited 116 packages in 0.23ms


In [82]:
import lightgbm as lgb

In [83]:
model1 = lgb.LGBMClassifier(random_state=20)
model1.fit(X_train, y_train)
test_pred1 = model1.predict(X_test)

[LightGBM] [Info] Number of positive: 815, number of negative: 3964
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001415 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3964
[LightGBM] [Info] Number of data points in the train set: 4779, number of used features: 60
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.170538 -> initscore=-1.581821
[LightGBM] [Info] Start training from score -1.581821


In [84]:
accuracy_score(y_test, test_pred1)

0.8545632015617375

In [ ]:
model1.predict_proba(X_test)

array([[0.99877048, 0.00122952],
       [0.94727742, 0.05272258],
       [0.40528058, 0.59471942],
       ...,
       [0.99310533, 0.00689467],
       [0.96358578, 0.03641422],
       [0.96991417, 0.03008583]], shape=(2049, 2))